In [697]:
import pandas as pd
import numpy as np
import random as rand
import os
from collections import Counter

In [698]:
cd ../data

/Users/johnwalecka/Desktop/Springboard/march_madness/march_madness_repo/data


In [699]:
tourney_teams = pd.read_csv('MNCAATourneySeeds.csv', index_col ='Season').loc['2015'].reset_index()
tourney_results = pd.read_csv('MNCAATourneyCompactResults.csv', index_col ='Season').loc['2015', ['WTeamID', 'LTeamID']].reset_index()

In [700]:
sim_teams = tourney_teams.copy()
true_teams = tourney_teams.copy()
sim_results = list()
true_results = list()

In [701]:
def reset_dataframes():
    sim_teams = tourney_teams.copy()
    true_teams = tourney_teams.copy()
    sim_results = list()
    true_results = list()

In [702]:
def get_play_in_teams(df_seeds):
    bool_array = (df_seeds['Seed'].str.find('a') != -1) | (df_seeds['Seed'].str.find('b') != -1)
    return true_teams[bool_array]

In [703]:
def play_game(team_1, team_2):
    if rand.uniform(0, 1) > 0.5:
        return team_1
    else:
        return team_2    

In [704]:
def first_team_wins(team_1_id, team_2_id):
    for index, row in tourney_results.iterrows():
        winner_id = tourney_results.iloc[index]['WTeamID']
        loser_id = tourney_results.iloc[index]['LTeamID']
        if ((winner_id == team_1_id) and (loser_id == team_2_id)):
            return True
    return False

In [705]:
def is_play_in_team(seed):
    str_len = len(seed)
    return ((seed[str_len - 1] == 'a') or (seed[str_len - 1] == 'b'))

In [706]:
def set_region(region_series):
    team_list = list()
    region_list = list(region_series)
    team_list.append(region_list[0])
    team_list.append(region_list[15])
    team_list.append(region_list[7])
    team_list.append(region_list[8])
    team_list.append(region_list[4])
    team_list.append(region_list[11])
    team_list.append(region_list[3])
    team_list.append(region_list[12])
    team_list.append(region_list[5])
    team_list.append(region_list[10])
    team_list.append(region_list[2])
    team_list.append(region_list[13])
    team_list.append(region_list[6])
    team_list.append(region_list[9])
    team_list.append(region_list[1])
    team_list.append(region_list[14])
    return team_list
    

In [707]:
def true_teams_index(team_id):
    return true_teams.index[true_teams['TeamID'] == team_id].tolist()[0]

In [708]:
def sim_teams_index(team_id):
    bool_array = sim_teams['TeamID'] == team_id
    print(len(bool_array))
    print(sim_teams)
    return sim_teams.index[bool_array].tolist()[0]

In [709]:
def play_play_in_games():
    global true_teams
    global true_results
    play_in_teams = get_play_in_teams(true_teams)
    iters = len(play_in_teams)
    drop_indices = list()
    for index in range(0, iters, 2):
        team_1_id = play_in_teams.iloc[index]['TeamID']
        team_2_id = play_in_teams.iloc[index + 1]['TeamID']
        if first_team_wins(team_1_id, team_2_id):
            drop_index = true_teams_index(team_2_id)
            drop_indices.append(drop_index)
        if first_team_wins(team_2_id, team_1_id):
            drop_index = true_teams_index(team_1_id)
            drop_indices.append(drop_index)
    true_teams = true_teams.drop(drop_indices)
    true_teams = true_teams.reset_index(drop=True)
    true_results.extend(list(play_in_teams['TeamID']))

In [710]:
def sim_play_in_games():
    global sim_teams
    global sim_results
    play_in_teams = get_play_in_teams(sim_teams)
    iters = len(play_in_teams)
    drop_indices = list()
    for index in range(0, iters, 2):
        team_1_id = play_in_teams.iloc[index]['TeamID']
        team_2_id = play_in_teams.iloc[index + 1]['TeamID']
        if play_game(team_1_id, team_2_id) == team_1_id:
            drop_index = sim_teams_index(team_2_id)
            drop_indices.append(drop_index)
        else:
            drop_index = sim_teams_index(team_1_id)
            drop_indices.append(drop_index)
    sim_teams = sim_teams.drop(drop_indices)
    sim_teams = sim_teams.reset_index(drop=True)
    sim_results.extend(list(play_in_teams['TeamID']))

In [711]:
def get_field_64(df_teams):
    team_ids = df_teams['TeamID']
    teams = list()
    region_1 = set_region(team_ids.iloc[0:16])
    region_2 = set_region(team_ids.iloc[16:32])
    region_3 = set_region(team_ids.iloc[32:48])
    region_4 = set_region(team_ids.iloc[48:64])
    teams.extend(region_1)
    teams.extend(region_2)
    teams.extend(region_3)
    teams.extend(region_4)
    return teams

In [712]:
def sim_round(team_list):
    num_teams = len(team_list)
    results = list()
    curr_index = int()
    while len(results) < (num_teams/2):
        winner = play_game(team_list[curr_index], team_list[curr_index + 1])
        results.append(winner)
        curr_index = curr_index + 2
    return results       

In [713]:
def sim_tourney():
    global sim_teams
    global sim_results
    sim_play_in_games()
    sim_field = get_field_64(sim_teams)
    sim_results.extend(sim_field)
    while len(sim_field) > 1:
        sim_field = sim_round(sim_field)
        sim_results.extend(sim_field)

In [714]:
def play_tourney():
    global true_teams
    global true_results
    total_games = 63 #total games from field of 64 onwards
    curr_index = 8 #start at index after play-in games
    play_play_in_games()
    true_field = get_field_64(true_teams)
    true_results.extend(true_field)
    for index in range(total_games):
        team_1_id = true_results[curr_index]
        team_2_id = true_results[curr_index + 1]
        if first_team_wins(team_1_id, team_2_id):
            true_results.append(team_1_id)
        elif first_team_wins(team_2_id, team_1_id):
            true_results.append(team_2_id)
        curr_index = curr_index + 2

In [715]:
def bracket_score(results, simulation):
    set_teams = 68
    score = int()
    for index in range(len(results) - 1):
        if (results[index] == simulation[index]):
            score = score + 1
    return (score - set_teams)
            

In [716]:
scores = list()
for i in range(5):
    reset_dataframes()
    sim_tourney()
    play_tourney()
    score = bracket_score(sim_results, true_results)
    scores.append(score)
    print(scores)

68
    Season Seed  TeamID
0     2015  W01    1437
1     2015  W02    1438
2     2015  W03    1328
3     2015  W04    1257
4     2015  W05    1320
..     ...  ...     ...
63    2015  Z12    1459
64    2015  Z13    1217
65    2015  Z14    1209
66    2015  Z15    1411
67    2015  Z16    1157

[68 rows x 3 columns]
68
    Season Seed  TeamID
0     2015  W01    1437
1     2015  W02    1438
2     2015  W03    1328
3     2015  W04    1257
4     2015  W05    1320
..     ...  ...     ...
63    2015  Z12    1459
64    2015  Z13    1217
65    2015  Z14    1209
66    2015  Z15    1411
67    2015  Z16    1157

[68 rows x 3 columns]
68
    Season Seed  TeamID
0     2015  W01    1437
1     2015  W02    1438
2     2015  W03    1328
3     2015  W04    1257
4     2015  W05    1320
..     ...  ...     ...
63    2015  Z12    1459
64    2015  Z13    1217
65    2015  Z14    1209
66    2015  Z15    1411
67    2015  Z16    1157

[68 rows x 3 columns]
68
    Season Seed  TeamID
0     2015  W01    1437
1     2

IndexError: list index out of range

In [ ]:
print(scores)